# Classify Images on Webcam. 
### Dependencies Library

 - library : OpenWeb Camera 

In [1]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode, b64encode

 - library : Image Processing & Ai tools 

In [2]:
from keras.models import load_model
from PIL import Image, ImageOps  
import numpy as np 
import io
import cv2

#### Import File Model Teachable .h5

In [3]:
import zipfile
from google.colab import files

%cd /content
uploaded = files.upload()   
for filename in uploaded.keys():
  pass

zip_ref = zipfile.ZipFile(filename, 'r')
zip_ref.extractall("/content/")

/content


Saving converted_keras.zip to converted_keras.zip


#### Function OpenWebcame on Colab 

In [4]:
def VideoCapture():
  js = Javascript('''
    async function create(){
      div = document.createElement('div');
      document.body.appendChild(div);

      video = document.createElement('video');
      video.setAttribute('playsinline', '');

      div.appendChild(video);

      stream = await navigator.mediaDevices.getUserMedia({video: {facingMode: "environment"}});
      video.srcObject = stream;

      await video.play();

      canvas =  document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);

      div_out = document.createElement('div');
      document.body.appendChild(div_out);
      img = document.createElement('img');
      div_out.appendChild(img);
    }

      async function capture() {
          canvas.width = video.videoWidth;  // Use half of the video resolution
          canvas.height = video.videoHeight;
          canvas.getContext('2d').drawImage(video, 0, 0, canvas.width, canvas.height); // Use the resized canvas for drawing the image
          const result = canvas.toDataURL('image/jpeg', 0.8); // Reduce the image quality to 50%
          return result;
      }

    function showimg(imgb64){
        img.src = "data:image/jpg;base64," + imgb64;
    }

  ''')
  display(js)

def byte2image(byte):
  jpeg = b64decode(byte.split(',')[1])
  im = Image.open(io.BytesIO(jpeg))
  return np.array(im)

def image2byte(image):
  image = Image.fromarray(image)
  buffer = io.BytesIO()
  image.save(buffer, 'jpeg')
  buffer.seek(0)
  x = b64encode(buffer.read()).decode('utf-8')
  return x

#### Task Image Processing & Ai on Webcame

In [8]:
VideoCapture()
eval_js('create()')
np.set_printoptions(suppress=True)
model = load_model("/content/keras_model.h5", compile=False)
class_names = open("/content/labels.txt", "r").readlines()
while True:
  byte = eval_js('capture()')
  data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
  frame = byte2image(byte)
  image = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
  size = (224, 224)
  image = cv2.resize(image,size)
  # turn the image into a numpy array
  image_array = np.asarray(image)

  # Normalize the image
  normalized_image_array = (image_array.astype(np.float32) / 127.5) - 1

  # Load the image into the array
  data[0] = normalized_image_array

  # Predicts the model
  prediction = model.predict(data,verbose=0)
  index = np.argmax(prediction)
  class_name = class_names[index]
  confidence_score = prediction[0][index]
  cv2.putText(frame,"class: "+str(class_name).replace('\n', ''), (0,25), cv2.FONT_HERSHEY_COMPLEX, .7, (255,255,255),2)
  cv2.putText(frame,"score: "+str(confidence_score), (0,50), cv2.FONT_HERSHEY_COMPLEX, .7, (255,255,255),2)
  eval_js('showimg("{}")'.format(image2byte(frame)))

<IPython.core.display.Javascript object>

KeyboardInterrupt: ignored